In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torchvision.models as models
import time

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Data augmentation for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomCrop(32, padding=4),  # Randomly crop the image
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No augmentation for validation and test
val_test_transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10 dataset
train_val_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=val_test_transform)

# Split train_val_dataset into train and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

# Apply val_test_transform to the validation set
val_dataset.dataset.transform = val_test_transform

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

100%|██████████| 170M/170M [00:02<00:00, 59.1MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# Load pretrained ResNet-50 (Teacher Model)
teacher = models.resnet50(pretrained=True)

# Modify the final fully connected layer for 10 classes (CIFAR-10)
teacher.fc = nn.Linear(teacher.fc.in_features, 10)
# Move models to device
teacher = teacher.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 198MB/s] 


In [5]:

model_path = '/kaggle/input/best_teacher/pytorch/default/1/Best_Teacher.pth'
# Load the model weights
teacher.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<ipython-input-5-79fc1ac30e55>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


<All keys matched successfully>

In [6]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet34(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 10)
student = student.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 114MB/s] 


In [7]:
def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total
    

In [8]:
def calculate_sparsity(model):
    total_zeros = 0
    total_params = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_zeros += torch.sum(param == 0).item()
            total_params += param.numel()
    return total_zeros / total_params

In [9]:
def measure_inference_time(model, test_loader, device, num_runs=5):
    model.eval()
    model.to(device)
    
    # Warm-up (to avoid initial overhead)
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            _ = model(inputs)
            break  # Only one batch for warm-up
    
    # Measure inference time
    total_time = 0
    with torch.no_grad():
        for _ in range(num_runs):
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                start_time = time.time()  # Start timer
                _ = model(inputs)
                end_time = time.time()  # End timer
                total_time += (end_time - start_time)
    
    # Average inference time per batch
    avg_time_per_batch = total_time / (num_runs * len(test_loader))
    return avg_time_per_batch

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def calculate_model_size(model, filename="temp.pth"):
    torch.save(model.state_dict(), filename)
    size = os.path.getsize(filename) / (1024 * 1024)  # Size in MB
    os.remove(filename)
    return size

def compare_model_sizes(teacher, student, pruned_student):
    # Count parameters
    teacher_params = count_parameters(teacher)
    student_params = count_parameters(student)
    pruned_params = count_parameters(pruned_student)
    
    # Calculate disk size
    teacher_size = calculate_model_size(teacher, "teacher.pth")
    student_size = calculate_model_size(student, "student.pth")
    pruned_size = calculate_model_size(pruned_student, "pruned_student.pth")
    
    # Print comparison
    print("\n--- Model Size Comparison ---")
    print(f"Teacher Model: {teacher_params} parameters, {teacher_size:.2f} MB")
    print(f"Student Model (Before Pruning): {student_params} parameters, {student_size:.2f} MB")
    print(f"Student Model (After Pruning): {pruned_params} parameters, {pruned_size:.2f} MB")
    
    # Calculate compression ratio
    compression_ratio = student_size / pruned_size
    print(f"\nCompression Ratio: {compression_ratio:.2f}x")

In [11]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001, patience=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    best_val_accuracy = 0.0
    best_model_state = None
    patience_counter = 0  # Counter for early stopping
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Evaluate on the validation set
        val_accuracy = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Val Accuracy: {val_accuracy:.2f}%")
        
        # Early stopping logic
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            patience_counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_teacher_model.pth')  # Save the best model
            print(f" New best model saved with validation accuracy: {best_val_accuracy:.2f}%")
        else:
            patience_counter += 1
            print(f" No improvement in validation accuracy ({patience_counter}/{patience})")
            
            # Stop training if no improvement for 'patience' epochs
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered! No improvement for {patience} epochs.")
                break
    
    # Load the best model state
    model.load_state_dict(torch.load('best_teacher_model.pth'))
    print("\nLoading the best model for final evaluation.")
    
    # Evaluate on the test set
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Test Accuracy with Best Model: {test_accuracy:.2f}%")
    
    return model



In [13]:
# Fine-tune the teacher model
teacher = train_model(teacher, train_loader, val_loader, epochs=200, lr=0.01, patience=10)

Epoch 1/200 | Train Loss: 3.2385 | Train Acc: 21.75% | Val Loss: 2.8543 | Val Acc: 29.15%
New best model saved with validation accuracy: 29.15%
Epoch 2/200 | Train Loss: 2.8950 | Train Acc: 27.93% | Val Loss: 2.7926 | Val Acc: 31.80%
New best model saved with validation accuracy: 31.80%
Epoch 3/200 | Train Loss: 2.5480 | Train Acc: 34.21% | Val Loss: 2.5264 | Val Acc: 37.16%
New best model saved with validation accuracy: 37.16%
Epoch 4/200 | Train Loss: 2.3018 | Train Acc: 39.28% | Val Loss: 2.5428 | Val Acc: 38.85%
New best model saved with validation accuracy: 38.85%
Epoch 5/200 | Train Loss: 2.1294 | Train Acc: 43.27% | Val Loss: 2.3666 | Val Acc: 39.74%
New best model saved with validation accuracy: 39.74%
Epoch 6/200 | Train Loss: 1.9874 | Train Acc: 46.10% | Val Loss: 2.1242 | Val Acc: 44.01%
New best model saved with validation accuracy: 44.01%
Epoch 7/200 | Train Loss: 1.9121 | Train Acc: 48.23% | Val Loss: 4.1372 | Val Acc: 22.39%
No improvement in validation accuracy (1/10)
E

KeyboardInterrupt: 

In [12]:

def compute_gradient_importance(teacher, student, data_loader, device, temperature=4.0, alpha=0.5):
    importance_scores = {}
    
    # Initialize importance storage for all Conv2d weights
    for name, param in student.named_parameters():
        if 'weight' in name and isinstance(param, nn.Parameter) and len(param.shape) == 4:
            importance_scores[name] = torch.zeros_like(param.data, device=device)
    
    teacher.to(device)
    student.to(device)
    
    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        student.zero_grad()
        
        with torch.no_grad():
            # Apply temperature scaling to teacher logits
            teacher_logits = teacher(inputs)
            teacher_probs = F.softmax(teacher_logits / temperature, dim=1)
            
        # Apply temperature scaling to student logits
        student_logits = student(inputs)
        student_probs = F.softmax(student_logits / temperature, dim=1)
        
        # Compute KL divergence loss (distillation loss)
        kl_loss = F.kl_div(
            F.log_softmax(student_logits / temperature, dim=1),
            teacher_probs,
            reduction='batchmean'
        ) * (temperature ** 2)  # Scale by temperature squared
        
        # Compute log-likelihood loss (ground truth alignment)
        log_likelihood_loss = F.cross_entropy(student_logits, labels)
        
        # Combine losses
        loss = alpha * kl_loss + (1 - alpha) * log_likelihood_loss
        loss.backward()
        
        # Accumulate absolute gradients for weights
        for name, param in student.named_parameters():
            if name in importance_scores and param.grad is not None:
                importance_scores[name] += param.grad.abs().detach()
    
    # Average across batches
    for name in importance_scores:
        importance_scores[name] /= len(data_loader)
        
    return importance_scores

In [13]:
def gradient_based_unstructured_prune(model, importance_scores, prune_ratio=0.3):
    """
    True unstructured pruning using per-weight importance scores
    """
    for name, param in model.named_parameters():
        if name in importance_scores:
            scores = importance_scores[name]
            n_prune = int(prune_ratio * scores.numel())
            
            if n_prune > 0:
                # Flatten and find threshold
                flat_scores = scores.flatten()
                threshold = torch.topk(flat_scores, k=n_prune, largest=False)[0][-1]
                
                # Create mask and apply pruning
                mask = (scores > threshold).float()
                param.data.mul_(mask)
    
    return model

def prune_model(model, importance_scores, prune_ratio=0.3, pruning_type='structured'):
    """
    Prune the model using either structured or unstructured pruning.
    """
    if pruning_type == 'structured':
        return channel_prune(model, importance_scores, prune_ratio)
    elif pruning_type == 'unstructured':
        return gradient_based_unstructured_prune(model, importance_scores, prune_ratio)
    else:
        raise ValueError("Invalid pruning type. Choose 'structured' or 'unstructured'.")

In [14]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

def train_kd_pruning(teacher, student, train_loader, val_loader, epochs=50, pruning_type='unstructured', temperature=5.0, alpha=0.5, patience=5, save_path="student_before_pruning.pth"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)
    
    teacher = teacher.to(device)
    student = student.to(device)
    
    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()
    
    # Training loop
    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            with torch.no_grad():
                # Apply temperature scaling to teacher logits
                teacher_logits = teacher(inputs)
                teacher_probs = F.softmax(teacher_logits / temperature, dim=1)
            
            # Apply temperature scaling to student logits
            student_logits = student(inputs)
            student_probs = F.log_softmax(student_logits / temperature, dim=1)
            
            # Compute distillation loss (KL divergence)
            distillation_loss = F.kl_div(student_probs, teacher_probs, reduction='batchmean') * (temperature ** 2)
            
            # Compute ground truth loss (cross-entropy)
            ground_truth_loss = F.cross_entropy(student_logits, labels)
            
            # Combine losses using alpha
            loss = alpha * distillation_loss + (1 - alpha) * ground_truth_loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
        
        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total
        
        # Validation
        val_acc = evaluate(student, val_loader, device)
        
        # Print metrics
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Acc: {val_acc:.2f}%")
        
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break
    
    # Load the best model state
    student.load_state_dict(best_model_state)
    
    # Save the student model before pruning
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")
    # Print total training time
    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")
    return student

In [37]:
# # Train the student model using KD and pruning
student = train_kd_pruning(
    teacher, student, train_loader, val_loader,
    epochs=50, pruning_type='unstructured', temperature=5.0, alpha=0.5, patience=5,save_path="student_before_pruning.pth"
)

Epoch 1/50 | Train Loss: 0.8813 | Train Acc: 91.04% | Val Acc: 94.23%
Epoch 2/50 | Train Loss: 0.3009 | Train Acc: 97.47% | Val Acc: 96.16%
Epoch 3/50 | Train Loss: 0.1924 | Train Acc: 99.03% | Val Acc: 96.61%
Epoch 4/50 | Train Loss: 0.1450 | Train Acc: 99.44% | Val Acc: 96.83%
Epoch 5/50 | Train Loss: 0.1202 | Train Acc: 99.53% | Val Acc: 96.97%
Epoch 6/50 | Train Loss: 0.1049 | Train Acc: 99.55% | Val Acc: 96.84%
Epoch 7/50 | Train Loss: 0.0930 | Train Acc: 99.60% | Val Acc: 96.94%
Epoch 8/50 | Train Loss: 0.0849 | Train Acc: 99.57% | Val Acc: 96.92%
Epoch 9/50 | Train Loss: 0.0792 | Train Acc: 99.61% | Val Acc: 97.08%
Epoch 10/50 | Train Loss: 0.0735 | Train Acc: 99.62% | Val Acc: 97.05%
Epoch 11/50 | Train Loss: 0.0698 | Train Acc: 99.64% | Val Acc: 97.00%
Epoch 12/50 | Train Loss: 0.0663 | Train Acc: 99.64% | Val Acc: 97.06%
Epoch 13/50 | Train Loss: 0.0637 | Train Acc: 99.64% | Val Acc: 97.07%
Epoch 14/50 | Train Loss: 0.0624 | Train Acc: 99.61% | Val Acc: 96.97%
Early stopping 

In [38]:
# Measure inference times
teacher_inference_time = measure_inference_time(teacher, test_loader, device)
student_inference_time = measure_inference_time(student, test_loader, device)
print(f"Teacher Model Inference Time: {teacher_inference_time * 1000:.2f} ms per batch")
print(f"Student Model Inference Time(Before Pruning): {student_inference_time * 1000:.2f} ms per batch")

Teacher Model Inference Time: 7.43 ms per batch
Student Model Inference Time(Before Pruning): 6.04 ms per batch


In [39]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 95.31%
Student Model Test Accuracy Before Pruning: 96.55%


In [40]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()

importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.5
)

total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = prune_model(student, importance_scores, prune_ratio=0.9,pruning_type='unstructured')

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Total Time take to calculate Important scores: 2m 31s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [41]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")
sparsity = calculate_sparsity(pruned_student)
print(f"Sparsity After Pruning: {sparsity * 100:.2f}%")

Student Model Test Accuracy After Pruning: 10.00%
Sparsity After Pruning: 89.94%


In [42]:
# Measure inference times
pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
print(f"Student Model Inference Time(After Pruning): {pruned_student_inference_time * 1000:.2f} ms per batch")

Student Model Inference Time(After Pruning): 5.97 ms per batch


In [43]:
torch.save(pruned_student.state_dict(), "pruned_student_unstructured_90%_res34.pth")
print("Model saved as pruned_student_unstructured.pth")


Model saved as pruned_student_unstructured.pth


In [15]:
import torch
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def retrain_with_sparsity(student, train_loader, val_loader, epochs=5, save_path="retrained_student_model.pt", patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks AND zero momentum buffers for pruned weights
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:  # Consider only conv layers
            mask = (param != 0).float().to(device)
            masks[name] = mask
            # Zero momentum buffers for pruned weights
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    student = student.to(device)
    best_val_acc = 0.0
    best_model = None
    patience_counter = 0  # Counter for early stopping

    # 2. Add gradient clipping to prevent NaN
    max_grad_norm = 1.0

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = student(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Apply masks to gradients
            for name, param in student.named_parameters():
                if name in masks:
                    param.grad.data *= masks[name]

            # Gradient clipping before optimizer step
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_grad_norm)

            optimizer.step()

            # Reapply masks and update momentum buffers
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation phase
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total

        # Track best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = student.state_dict()
            torch.save(best_model, save_path)
            patience_counter = 0  # Reset patience counter
            print(f"New best model saved with Val Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break  # Stop training

        # Print results
        sparsity = calculate_sparsity(student)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Validation Loss: {val_loss:.4f} | Validation Acc: {val_acc:.2f}% | Sparsity: {sparsity*100:.2f}%\n")

    print(f"Best Validation Accuracy: {best_val_acc:.2f}% | Best Model Saved at: {save_path}")
    return student

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

def retrain_with_KD(teacher, student, train_loader, val_loader, epochs=50, temperature=5.0, alpha=0.5, patience=5, save_path="student_before_pruning.pth"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)
     # 1. Store masks AND zero momentum buffers for pruned weights
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:  # Consider only conv layers
            mask = (param != 0).float().to(device)
            masks[name] = mask
            # Zero momentum buffers for pruned weights
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask
    
    teacher = teacher.to(device)
    student = student.to(device)
    
    best_val_acc = 0.0
    best_val_loss = float("inf")
    best_model_state = None
    patience_counter = 0
    start_time = time.time()
    
    # Training loop
    for epoch in range(epochs):
        student.train()
        total_loss, correct, total = 0.0, 0, 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            with torch.no_grad():
                # Apply temperature scaling to teacher logits
                teacher_logits = teacher(inputs)
                teacher_probs = F.softmax(teacher_logits / temperature, dim=1)
            
            # Apply temperature scaling to student logits
            student_logits = student(inputs)
            student_probs = F.log_softmax(student_logits / temperature, dim=1)
            
            # Compute distillation loss (KL divergence)
            distillation_loss = F.kl_div(student_probs, teacher_probs, reduction='batchmean') * (temperature ** 2)
            
            # Compute ground truth loss (cross-entropy)
            ground_truth_loss = F.cross_entropy(student_logits, labels)
            
            # Combine losses using alpha
            loss = alpha * distillation_loss + (1 - alpha) * ground_truth_loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Reapply masks and update momentum buffers
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]
            
            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
        
        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total
        
        # Validation
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)
        
        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total
        sparsity = calculate_sparsity(student)*100.0  # Function assumed to be defined
        
        # Print metrics
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Sparsity: {sparsity:.2f}%")
        
        # Early stopping
        if val_acc > best_val_acc or val_loss < best_val_loss:
            best_val_acc = val_acc
            best_val_loss = val_loss
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break
    
    # Load the best model state
    student.load_state_dict(best_model_state)
    
    # Save the student model before pruning
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")
    
    # Print total training time
    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")
    
    return student


# 90% of Sparsity

In [46]:
student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='/kaggle/working/retrained_student_model.pt',patience=10
)

New best model saved with Val Accuracy: 35.90%
Epoch 1/200 | Train Loss: 1.9981 | Train Acc: 24.29%
Validation Loss: 1.6694 | Validation Acc: 35.90% | Sparsity: 89.94%

New best model saved with Val Accuracy: 43.64%
Epoch 2/200 | Train Loss: 1.5134 | Train Acc: 42.51%
Validation Loss: 1.6567 | Validation Acc: 43.64% | Sparsity: 89.94%

New best model saved with Val Accuracy: 52.91%
Epoch 3/200 | Train Loss: 1.1877 | Train Acc: 56.86%
Validation Loss: 1.2756 | Validation Acc: 52.91% | Sparsity: 89.94%

New best model saved with Val Accuracy: 64.16%
Epoch 4/200 | Train Loss: 0.9905 | Train Acc: 64.30%
Validation Loss: 1.0049 | Validation Acc: 64.16% | Sparsity: 89.94%

New best model saved with Val Accuracy: 69.85%
Epoch 5/200 | Train Loss: 0.8679 | Train Acc: 68.94%
Validation Loss: 0.8399 | Validation Acc: 69.85% | Sparsity: 89.94%

Epoch 6/200 | Train Loss: 0.7788 | Train Acc: 72.17%
Validation Loss: 0.8398 | Validation Acc: 69.71% | Sparsity: 89.94%

New best model saved with Val Acc

In [78]:
student_accuracy = evaluate(student, test_loader, device)
print(f"Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy: 92.71%


In [79]:
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Sparsity: 80.95%


In [80]:

pruned_student_inference_time = measure_inference_time(student, test_loader, device)
print(f"Pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")


Pruned Student Model Inference Time: 6.02 ms per batch


# Retrain with KD

In [36]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet34(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 10)
student = student.to(device)

In [39]:

model_path = '/kaggle/input/student_resnet_34/pytorch/default/1/student_before_pruning_Res34.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<ipython-input-39-5d6657d12f2d>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


<All keys matched successfully>

In [58]:

# model_path = '/kaggle/working/pruned_student_unstructured_90%_res34.pth'
# # Load the model weights
# student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<ipython-input-58-0660dd01e2d9>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


<All keys matched successfully>

In [40]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.5
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = prune_model(student, importance_scores, prune_ratio=0.9,pruning_type='unstructured')

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Total Time take to calculate Important scores: 2m 33s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [41]:
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.5, patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)

Epoch 1/50 | Train Loss: 7.5166 | Train Acc: 18.06% | Val Loss: 1.8806 | Val Acc: 30.67% | Sparsity: 89.94%
Epoch 2/50 | Train Loss: 5.1925 | Train Acc: 41.24% | Val Loss: 1.8175 | Val Acc: 48.11% | Sparsity: 89.94%
Epoch 3/50 | Train Loss: 3.5202 | Train Acc: 62.22% | Val Loss: 1.2330 | Val Acc: 65.08% | Sparsity: 89.94%
Epoch 4/50 | Train Loss: 2.6214 | Train Acc: 72.11% | Val Loss: 1.1596 | Val Acc: 71.92% | Sparsity: 89.94%
Epoch 5/50 | Train Loss: 2.0768 | Train Acc: 78.52% | Val Loss: 0.9063 | Val Acc: 75.79% | Sparsity: 89.94%
Epoch 6/50 | Train Loss: 1.7329 | Train Acc: 82.24% | Val Loss: 0.9131 | Val Acc: 77.25% | Sparsity: 89.94%
Epoch 7/50 | Train Loss: 1.5006 | Train Acc: 84.49% | Val Loss: 0.8240 | Val Acc: 79.17% | Sparsity: 89.94%
Epoch 8/50 | Train Loss: 1.3070 | Train Acc: 86.61% | Val Loss: 0.6317 | Val Acc: 83.51% | Sparsity: 89.94%
Epoch 9/50 | Train Loss: 1.1481 | Train Acc: 88.51% | Val Loss: 0.8030 | Val Acc: 80.09% | Sparsity: 89.94%
Epoch 10/50 | Train Loss: 1.

In [42]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy: 88.55%


In [59]:
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Sparsity: 89.94%


# 80% of Sparsity

In [31]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet34(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 10)
student = student.to(device)

In [32]:

model_path = '/kaggle/input/student_resnet_34/pytorch/default/1/student_before_pruning_Res34.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<ipython-input-32-5d6657d12f2d>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


<All keys matched successfully>

In [41]:
# Measure inference times
teacher_inference_time = measure_inference_time(teacher, test_loader, device)
student_inference_time = measure_inference_time(student, test_loader, device)
print(f"Teacher Model Inference Time: {teacher_inference_time * 1000:.2f} ms per batch")
print(f"Student Model Inference Time(Before Pruning): {student_inference_time * 1000:.2f} ms per batch")

Teacher Model Inference Time: 0.06 ms per batch
Student Model Inference Time(Before Pruning): 0.03 ms per batch


In [70]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 95.31%
Student Model Test Accuracy Before Pruning: 96.55%


In [33]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.5
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = prune_model(student, importance_scores, prune_ratio=0.81,pruning_type='unstructured')
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Total Time take to calculate Important scores: 2m 33s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [24]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")
sparsity = calculate_sparsity(pruned_student)
print(f"Sparsity After Pruning: {sparsity * 100:.2f}%")

Student Model Test Accuracy After Pruning: 10.00%
Sparsity After Pruning: 80.95%


In [75]:
torch.save(pruned_student.state_dict(), "pruned_student_unstructured_80%_res34.pth")
print("Model saved as pruned_student_unstructured.pth")


Model saved as pruned_student_unstructured.pth


In [76]:
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='/kaggle/working/retrained_student_model.pt',patience=10
)

New best model saved with Val Accuracy: 59.97%
Epoch 1/200 | Train Loss: 1.3459 | Train Acc: 48.86%
Validation Loss: 1.1134 | Validation Acc: 59.97% | Sparsity: 80.95%

New best model saved with Val Accuracy: 69.04%
Epoch 2/200 | Train Loss: 0.9368 | Train Acc: 66.44%
Validation Loss: 0.8673 | Validation Acc: 69.04% | Sparsity: 80.95%

New best model saved with Val Accuracy: 75.37%
Epoch 3/200 | Train Loss: 0.6922 | Train Acc: 75.64%
Validation Loss: 0.7140 | Validation Acc: 75.37% | Sparsity: 80.95%

New best model saved with Val Accuracy: 81.20%
Epoch 4/200 | Train Loss: 0.5133 | Train Acc: 82.20%
Validation Loss: 0.5655 | Validation Acc: 81.20% | Sparsity: 80.95%

New best model saved with Val Accuracy: 85.31%
Epoch 5/200 | Train Loss: 0.3747 | Train Acc: 87.15%
Validation Loss: 0.4407 | Validation Acc: 85.31% | Sparsity: 80.95%

New best model saved with Val Accuracy: 86.48%
Epoch 6/200 | Train Loss: 0.2741 | Train Acc: 90.55%
Validation Loss: 0.4096 | Validation Acc: 86.48% | Spar

In [77]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 92.71%


In [81]:
# Measure inference times
pruned_student_inference_time = measure_inference_time(retrained_student, test_loader, device)
print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

 Retrained pruned Student Model Inference Time: 6.01 ms per batch


In [82]:
torch.save(retrained_student.state_dict(), "pruned_retrained_student_unstructured_80%.pth")
print("Model saved as pruned_student_unstructured.pth")


Model saved as pruned_student_unstructured.pth


# Retrain With KD

In [34]:
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.5, patience=5,save_path="pruned_student_retrain_KD_80%.pth"
)

Epoch 1/50 | Train Loss: 6.7128 | Train Acc: 26.93% | Val Loss: 2.0609 | Val Acc: 35.80% | Sparsity: 80.95%
Epoch 2/50 | Train Loss: 3.1727 | Train Acc: 66.32% | Val Loss: 0.8355 | Val Acc: 77.17% | Sparsity: 80.95%
Epoch 3/50 | Train Loss: 1.5575 | Train Acc: 83.73% | Val Loss: 0.5588 | Val Acc: 84.83% | Sparsity: 80.95%
Epoch 4/50 | Train Loss: 1.0419 | Train Acc: 89.58% | Val Loss: 0.4963 | Val Acc: 87.12% | Sparsity: 80.95%
Epoch 5/50 | Train Loss: 0.7580 | Train Acc: 93.16% | Val Loss: 0.4160 | Val Acc: 89.33% | Sparsity: 80.95%
Epoch 6/50 | Train Loss: 0.5649 | Train Acc: 95.55% | Val Loss: 0.4072 | Val Acc: 90.07% | Sparsity: 80.95%
Epoch 7/50 | Train Loss: 0.4459 | Train Acc: 97.14% | Val Loss: 0.3546 | Val Acc: 90.61% | Sparsity: 80.95%
Epoch 8/50 | Train Loss: 0.3572 | Train Acc: 98.38% | Val Loss: 0.3098 | Val Acc: 91.93% | Sparsity: 80.95%
Epoch 9/50 | Train Loss: 0.3078 | Train Acc: 98.87% | Val Loss: 0.2786 | Val Acc: 92.77% | Sparsity: 80.95%
Epoch 10/50 | Train Loss: 0.

In [35]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 93.28%


# 70% Sparsity

In [17]:
# Load pretrained ResNet-34 (Student Model)
student = models.resnet34(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 10)
student = student.to(device)

In [18]:

model_path = '/kaggle/input/student_resnet_34/pytorch/default/1/student_before_pruning_Res34.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<ipython-input-18-5d6657d12f2d>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


<All keys matched successfully>

In [25]:
# Measure inference times
teacher_inference_time = measure_inference_time(teacher, test_loader, device)
student_inference_time = measure_inference_time(student, test_loader, device)
print(f"Teacher Model Inference Time: {teacher_inference_time * 1000:.2f} ms per batch")
print(f"Student Model Inference Time(Before Pruning): {student_inference_time * 1000:.2f} ms per batch")

Teacher Model Inference Time: 7.57 ms per batch
Student Model Inference Time(Before Pruning): 6.03 ms per batch


In [19]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 95.41%
Student Model Test Accuracy Before Pruning: 96.55%


In [20]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.5
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = prune_model(student, importance_scores, prune_ratio=0.707,pruning_type='unstructured')
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Total Time take to calculate Important scores: 2m 31s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [21]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")
sparsity = calculate_sparsity(pruned_student)
print(f"Sparsity After Pruning: {sparsity * 100:.2f}%")

Student Model Test Accuracy After Pruning: 10.00%
Sparsity After Pruning: 70.65%


In [22]:
# Measure inference times
pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
print(f"Student Model Inference Time(After Pruning): {pruned_student_inference_time * 1000:.2f} ms per batch")

Student Model Inference Time(After Pruning): 5.95 ms per batch


In [23]:
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='/kaggle/working/retrained_student_model_70%.pt',patience=5
)

New best model saved with Val Accuracy: 52.21%
Epoch 1/200 | Train Loss: 1.8875 | Train Acc: 32.25%
Validation Loss: 1.3421 | Validation Acc: 52.21% | Sparsity: 70.65%

New best model saved with Val Accuracy: 91.50%
Epoch 2/200 | Train Loss: 0.5644 | Train Acc: 80.40%
Validation Loss: 0.2579 | Validation Acc: 91.50% | Sparsity: 70.65%

New best model saved with Val Accuracy: 93.10%
Epoch 3/200 | Train Loss: 0.1795 | Train Acc: 94.03%
Validation Loss: 0.2114 | Validation Acc: 93.10% | Sparsity: 70.65%

New best model saved with Val Accuracy: 93.94%
Epoch 4/200 | Train Loss: 0.0901 | Train Acc: 96.93%
Validation Loss: 0.1904 | Validation Acc: 93.94% | Sparsity: 70.65%

New best model saved with Val Accuracy: 94.28%
Epoch 5/200 | Train Loss: 0.0523 | Train Acc: 98.26%
Validation Loss: 0.1911 | Validation Acc: 94.28% | Sparsity: 70.65%

Epoch 6/200 | Train Loss: 0.0315 | Train Acc: 98.99%
Validation Loss: 0.2178 | Validation Acc: 94.04% | Sparsity: 70.65%

Epoch 7/200 | Train Loss: 0.0247 

In [24]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 93.55%


In [25]:
# Measure inference times
pruned_student_inference_time = measure_inference_time(retrained_student, test_loader, device)
print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

 Retrained pruned Student Model Inference Time: 5.89 ms per batch


In [33]:
torch.save(retrained_student.state_dict(), "pruned_retrained_student_unstructured_80%.pth")
print("Model saved as pruned_student_unstructured.pth")


Model saved as pruned_student_unstructured.pth


# Retrain with KD

In [26]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet34(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 10)
student = student.to(device)

In [27]:

model_path = '/kaggle/input/student_resnet_34/pytorch/default/1/student_before_pruning_Res34.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<ipython-input-27-5d6657d12f2d>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


<All keys matched successfully>

In [28]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.5
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = prune_model(student, importance_scores, prune_ratio=0.707,pruning_type='unstructured')
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Total Time take to calculate Important scores: 2m 33s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [29]:
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.5, patience=5,save_path="pruned_student_retrain_KD_80%.pth"
)

Epoch 1/50 | Train Loss: 4.7593 | Train Acc: 50.07% | Val Loss: 0.5529 | Val Acc: 85.22% | Sparsity: 70.65%
Epoch 2/50 | Train Loss: 0.8204 | Train Acc: 92.30% | Val Loss: 0.2575 | Val Acc: 92.50% | Sparsity: 70.65%
Epoch 3/50 | Train Loss: 0.5003 | Train Acc: 96.36% | Val Loss: 0.2004 | Val Acc: 93.96% | Sparsity: 70.65%
Epoch 4/50 | Train Loss: 0.3455 | Train Acc: 98.28% | Val Loss: 0.1458 | Val Acc: 95.59% | Sparsity: 70.65%
Epoch 5/50 | Train Loss: 0.2706 | Train Acc: 99.10% | Val Loss: 0.1482 | Val Acc: 95.45% | Sparsity: 70.65%
Epoch 6/50 | Train Loss: 0.2312 | Train Acc: 99.33% | Val Loss: 0.1271 | Val Acc: 95.97% | Sparsity: 70.65%
Epoch 7/50 | Train Loss: 0.2022 | Train Acc: 99.41% | Val Loss: 0.1407 | Val Acc: 95.84% | Sparsity: 70.65%
Epoch 8/50 | Train Loss: 0.1881 | Train Acc: 99.52% | Val Loss: 0.1220 | Val Acc: 96.02% | Sparsity: 70.65%
Epoch 9/50 | Train Loss: 0.1775 | Train Acc: 99.45% | Val Loss: 0.1239 | Val Acc: 96.20% | Sparsity: 70.65%
Epoch 10/50 | Train Loss: 0.

In [30]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 95.38%


# 50% Sparsity

In [17]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet34(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 10)
student = student.to(device)

In [18]:

model_path = '/kaggle/input/student_resnet_34/pytorch/default/1/student_before_pruning_Res34.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<ipython-input-18-5d6657d12f2d>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


<All keys matched successfully>

In [16]:
# Measure inference times
teacher_inference_time = measure_inference_time(teacher, test_loader, device)
student_inference_time = measure_inference_time(student, test_loader, device)
print(f"Teacher Model Inference Time: {teacher_inference_time * 1000:.2f} ms per batch")
print(f"Student Model Inference Time(Before Pruning): {student_inference_time * 1000:.2f} ms per batch")

Teacher Model Inference Time: 7.42 ms per batch
Student Model Inference Time(Before Pruning): 3.67 ms per batch


In [17]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 95.41%
Student Model Test Accuracy Before Pruning: 95.67%


In [18]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 95.41%
Student Model Test Accuracy Before Pruning: 95.67%


In [19]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.5
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = prune_model(student, importance_scores, prune_ratio=0.505,pruning_type='unstructured')
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Total Time take to calculate Important scores: 2m 39s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [57]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")
sparsity = calculate_sparsity(pruned_student)
print(f"Sparsity After Pruning: {sparsity * 100:.2f}%")

Student Model Test Accuracy After Pruning: 10.00%
Sparsity After Pruning: 50.47%


In [21]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 50.46%
Teacher Model Test Accuracy: 95.41%
Student Model Test Accuracy Before Pruning: 10.00%


In [23]:
# Measure inference times
pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
print(f"Student Model Inference Time(After Pruning): {pruned_student_inference_time * 1000:.2f} ms per batch")

Student Model Inference Time(After Pruning): 3.70 ms per batch


In [20]:
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='/kaggle/working/retrained_student_model_50%.pt',patience=5
)

New best model saved with Val Accuracy: 97.55%
Epoch 1/200 | Train Loss: 0.2492 | Train Acc: 91.97%
Validation Loss: 0.0762 | Validation Acc: 97.55% | Sparsity: 50.47%

Epoch 2/200 | Train Loss: 0.0402 | Train Acc: 98.74%
Validation Loss: 0.0758 | Validation Acc: 97.50% | Sparsity: 50.47%

New best model saved with Val Accuracy: 97.87%
Epoch 3/200 | Train Loss: 0.0150 | Train Acc: 99.53%
Validation Loss: 0.0688 | Validation Acc: 97.87% | Sparsity: 50.47%

New best model saved with Val Accuracy: 97.98%
Epoch 4/200 | Train Loss: 0.0078 | Train Acc: 99.77%
Validation Loss: 0.0695 | Validation Acc: 97.98% | Sparsity: 50.47%

New best model saved with Val Accuracy: 98.08%
Epoch 5/200 | Train Loss: 0.0051 | Train Acc: 99.89%
Validation Loss: 0.0681 | Validation Acc: 98.08% | Sparsity: 50.47%

New best model saved with Val Accuracy: 98.11%
Epoch 6/200 | Train Loss: 0.0023 | Train Acc: 99.95%
Validation Loss: 0.0679 | Validation Acc: 98.11% | Sparsity: 50.47%

New best model saved with Val Acc

In [21]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Student Model Test Accuracy After retraining: {student_accuracy:.2f}%")

Student Model Test Accuracy After retraining: 95.93%


# Retraining with KD

In [22]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet34(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 10)
student = student.to(device)

In [23]:

model_path = '/kaggle/input/student_resnet_34/pytorch/default/1/student_before_pruning_Res34.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<ipython-input-23-5d6657d12f2d>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


<All keys matched successfully>

In [24]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.5
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = prune_model(student, importance_scores, prune_ratio=0.505,pruning_type='unstructured')
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Total Time take to calculate Important scores: 2m 38s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [25]:
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.5, patience=5,save_path="pruned_student_retrain_KD_80%.pth"
)

Epoch 1/50 | Train Loss: 0.6577 | Train Acc: 93.51% | Val Loss: 0.0915 | Val Acc: 96.89% | Sparsity: 50.47%
Epoch 2/50 | Train Loss: 0.2328 | Train Acc: 98.49% | Val Loss: 0.0687 | Val Acc: 97.78% | Sparsity: 50.47%
Epoch 3/50 | Train Loss: 0.1589 | Train Acc: 99.28% | Val Loss: 0.0626 | Val Acc: 97.99% | Sparsity: 50.47%
Epoch 4/50 | Train Loss: 0.1247 | Train Acc: 99.49% | Val Loss: 0.0573 | Val Acc: 98.04% | Sparsity: 50.47%
Epoch 5/50 | Train Loss: 0.1069 | Train Acc: 99.57% | Val Loss: 0.0550 | Val Acc: 98.07% | Sparsity: 50.47%
Epoch 6/50 | Train Loss: 0.0957 | Train Acc: 99.53% | Val Loss: 0.0569 | Val Acc: 97.99% | Sparsity: 50.47%
Epoch 7/50 | Train Loss: 0.0871 | Train Acc: 99.63% | Val Loss: 0.0518 | Val Acc: 98.26% | Sparsity: 50.47%
Epoch 8/50 | Train Loss: 0.0805 | Train Acc: 99.60% | Val Loss: 0.0522 | Val Acc: 98.24% | Sparsity: 50.47%
Epoch 9/50 | Train Loss: 0.0762 | Train Acc: 99.64% | Val Loss: 0.0521 | Val Acc: 98.24% | Sparsity: 50.47%
Epoch 10/50 | Train Loss: 0.

In [26]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After retraining: {student_accuracy:.2f}%")

Student Model Test Accuracy After retraining: 96.22%


# Retrain with 20% Training Data

In [16]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 10)
student = student.to(device)

In [17]:

model_path = '/kaggle/input/best_student_resnet18/pytorch/default/1/student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<ipython-input-17-b666940780fb>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


<All keys matched successfully>

In [ ]:
# Measure inference times
teacher_inference_time = measure_inference_time(teacher, test_loader, device)
student_inference_time = measure_inference_time(student, test_loader, device)
print(f"Teacher Model Inference Time: {teacher_inference_time * 1000:.2f} ms per batch")
print(f"Student Model Inference Time(Before Pruning): {student_inference_time * 1000:.2f} ms per batch")

In [ ]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

In [ ]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

In [18]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.5
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = prune_model(student, importance_scores, prune_ratio=0.9,pruning_type='unstructured')
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Total Time take to calculate Important scores: 0m 32s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [19]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")
sparsity = calculate_sparsity(pruned_student)
print(f"Sparsity After Pruning: {sparsity * 100:.2f}%")

Student Model Test Accuracy After Pruning: 10.00%
Sparsity After Pruning: 89.92%


In [ ]:

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

In [ ]:
# Measure inference times
pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
print(f"Student Model Inference Time(After Pruning): {pruned_student_inference_time * 1000:.2f} ms per batch")

In [20]:
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms

# Data augmentation for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomCrop(32, padding=4),  # Randomly crop the image
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No augmentation for validation and test
val_test_transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10 dataset
train_val_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=val_test_transform)

# Split train_val_dataset into train (20%) and validation (20% of original)
train_size = int(0.2 * len(train_val_dataset))  # 20% of 50,000 = 10,000 samples
remaining_size = len(train_val_dataset) - train_size

# Use random_split to preserve class distribution
train_dataset, remaining_data = random_split(train_val_dataset, [train_size, remaining_size])

# Split remaining_data into validation (20% of original)
val_size = int(0.2 * len(train_val_dataset))  # 20% of 50,000 = 10,000 samples
_, val_dataset = random_split(remaining_data, [remaining_size - val_size, val_size])

# Apply val_test_transform to the validation set
val_dataset.dataset.transform = val_test_transform

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Print dataset sizes
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Files already downloaded and verified
Files already downloaded and verified
Train dataset size: 10000
Validation dataset size: 10000
Test dataset size: 10000


In [22]:
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='/kaggle/working/retrained_student_model_90%_20%_Train_Data.pt',patience=20
)

New best model saved with Val Accuracy: 19.90%
Epoch 1/200 | Train Loss: 2.1372 | Train Acc: 18.24%
Validation Loss: 2.1238 | Validation Acc: 19.90% | Sparsity: 89.92%

New best model saved with Val Accuracy: 21.76%
Epoch 2/200 | Train Loss: 1.9587 | Train Acc: 21.95%
Validation Loss: 1.9937 | Validation Acc: 21.76% | Sparsity: 89.92%

Epoch 3/200 | Train Loss: 1.9046 | Train Acc: 23.17%
Validation Loss: 2.1815 | Validation Acc: 20.42% | Sparsity: 89.92%

New best model saved with Val Accuracy: 27.65%
Epoch 4/200 | Train Loss: 1.8598 | Train Acc: 26.08%
Validation Loss: 1.8995 | Validation Acc: 27.65% | Sparsity: 89.92%

New best model saved with Val Accuracy: 30.07%
Epoch 5/200 | Train Loss: 1.7397 | Train Acc: 31.92%
Validation Loss: 1.8803 | Validation Acc: 30.07% | Sparsity: 89.92%

Epoch 6/200 | Train Loss: 1.6891 | Train Acc: 34.55%
Validation Loss: 2.2614 | Validation Acc: 25.26% | Sparsity: 89.92%

New best model saved with Val Accuracy: 32.22%
Epoch 7/200 | Train Loss: 1.6415 

In [24]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 75.18%
